In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle


Data prepartion

In [ ]:

pos_dir='/content/drive/MyDrive/ביואינפורמטיקה/שנה ג/ביואינפורמטיקה מבנית/HW4- epitopes/pos_A0201.txt'
neg_dir='/content/drive/MyDrive/ביואינפורמטיקה/שנה ג/ביואינפורמטיקה מבנית/HW4- epitopes/neg_A0201.txt'

pos_data=pd.read_csv(pos_dir, sep=' ', header=None)
pos_data['class']=np.ones(len(pos_data)) #add the class column

neg_data=pd.read_csv(neg_dir, sep=' ', header=None)
neg_data['class']=np.zeros(len(neg_data)) #add the class column

all_data= pd.concat([pos_data, neg_data]) #join the tables

all_data=all_data.sample(frac=1).reset_index(drop=True) #shuffle the data

features=all_data[0]
clss=all_data['class']


print(features)

0        RPVRGPLLR
1        EQMGQMLTR
2        RLFENLRML
3        RIKPIYNLY
4        FVIHDMETL
           ...    
27478    AVIQVGKEK
27479    EAFQTTISK
27480    VLSEQQNGL
27481    TTNDIVISK
27482    SLLRHLEKV
Name: 0, Length: 27483, dtype: object


Split the sequence to amino acids

In [ ]:
features=features.str.split('',n=9, expand=True) 
features=features.drop(0,axis=1)
print(features)

       1  2  3  4  5  6  7  8  9
0      R  P  V  R  G  P  L  L  R
1      E  Q  M  G  Q  M  L  T  R
2      R  L  F  E  N  L  R  M  L
3      R  I  K  P  I  Y  N  L  Y
4      F  V  I  H  D  M  E  T  L
...   .. .. .. .. .. .. .. .. ..
27478  A  V  I  Q  V  G  K  E  K
27479  E  A  F  Q  T  T  I  S  K
27480  V  L  S  E  Q  Q  N  G  L
27481  T  T  N  D  I  V  I  S  K
27482  S  L  L  R  H  L  E  K  V

[27483 rows x 9 columns]


Transpormation to numeric data

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe=OneHotEncoder(sparse=False)
num_features=ohe.fit_transform(features)
num_features=pd.DataFrame(num_features)  
print(num_features.shape)

(27483, 180)


Train-set and test-set splitting

In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_features = train_test_split(num_features, test_size=0.2, shuffle=False)
train_class, test_class=train_test_split(clss, test_size=0.2, shuffle=False)

print(train_features.shape)

(21986, 180)


Balance the groups

In [ ]:
from imblearn.over_sampling import RandomOverSampler 

ros = RandomOverSampler(sampling_strategy='minority')
features_resampled, class_resampled = ros.fit_resample(train_features, train_class)

print(features_resampled.shape)

(39106, 180)


Build and fit the NN

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report


clf = MLPClassifier(hidden_layer_sizes=(36,36,10),
                    activation='tanh',
                    solver='sgd',
                    alpha=0.01,
                    learning_rate='adaptive',
                    early_stopping=True,
                    validation_fraction=0.25,
                    )

clf.fit(features_resampled, class_resampled)



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPClassifier(activation='tanh', alpha=0.01, early_stopping=True,
              hidden_layer_sizes=(36, 36, 10), learning_rate='adaptive',
              solver='sgd', validation_fraction=0.25)

Scores

In [ ]:
print(clf.score(test_features, test_class))
pred_test_class=clf.predict(test_features)
print(classification_report(test_class, pred_test_class))

0.8732035655812261
              precision    recall  f1-score   support

         0.0       0.99      0.87      0.92      4939
         1.0       0.44      0.92      0.60       558

    accuracy                           0.87      5497
   macro avg       0.72      0.90      0.76      5497
weighted avg       0.93      0.87      0.89      5497

